In [26]:
import cv2 as cv
import numpy as np

fgbg = cv.createBackgroundSubtractorKNN(detectShadows=False)
    #MOG2: Gaussian mixture-based background/foreground segmentation algorithm. better results with morphology.
    #KNN: K-nearest neighbors background/foreground segmentation algorithm.

cap = cv.VideoCapture(r"C:\Users\asus\Downloads\volleyball_match.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame)

    height, width = frame.shape[:2]
    mask = np.zeros_like(fgmask)
    mask[:height // 2, :] = 255 #cutting vid in half since ball is in upper half for 80%

    fgmask_upper_half = cv.bitwise_and(fgmask, fgmask, mask=mask)



    _, thresholded = cv.threshold(fgmask_upper_half, 127, 255, cv.THRESH_BINARY)

    contours, _ = cv.findContours(thresholded, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv.contourArea(contour)
        if area < 100 or area > 700: 
            continue

        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv.imshow('Volleyball Tracking', frame)
    cv.imshow('Foreground Mask', fgmask_upper_half)

    if cv.waitKey(20) & 0xFF == 27:   # esc
        break

cap.release()
cv.destroyAllWindows()
